# LAB 4

# Georgia Zavou
# Ilaria Curzi

# 1. One simple way to proceed is to take  and iteratively update the current point to obtain the next. This is a simple way to proceed that is proposed to perform first. The stopping condition should be performed over . Test this approach and check if it works using the starting point proposed in the example.

Initially we define the function that returns f and the Hessian. Then functions that compute the gradient and hessians given the point x1 and x2. Following we define functions to compute the Lagrange values.

In [ ]:
from math import exp as e
import numpy as np


# Function f
def f(x_1,x_2):
  return e(3*x_1) + e(-4*x_2)

# Hessian
def h(x_1,x_2):
  return x_1**2 + x_2**2 - 1

# Gradient of  f
def grad_f(x_1, x_2):
    return np.array([3*e(3*x_1), -4*e(-4*x_2)])

# Gradient of  constrained function
def grad_h(x_1, x_2):
    return np.array([2*x_1, 2*x_2])

# Hessian of f
def hess_f(x_1, x_2):
    return np.array([[9*e(3*x_1), 0], [0, 16*e(-4*x_2)]])

# Hessian of  constrained function
def hess_h(x_1, x_2):
    return np.array([[2, 0], [0, 2]])

# Lagrange of f
def lag(x_1, x_2, lamda):
    return f(x_1, x_2) - lamda*h(x_1, x_2)

# Lagrange of gradient
def grad_lag(x_1, x_2, lamda):
    return grad_f(x_1, x_2) - lamda*grad_h(x_1, x_2)

# Lagrange of Hessian
def hess_lag(x_1, x_2, lamda):
    return hess_f(x_1, x_2) - lamda*hess_h(x_1, x_2)


Now we define a function using Newtons step to iterate until we find the solution.

In [ ]:
def solveNewtonBased(x_1, x_2, lamda, alpha=1, eps=1e-5, MAX_ITER=100):

    print(f'Starting point x0 = [{x_1}, {x_2}]')

    for i in range(MAX_ITER):
        # Compute the gradient of the constraint function h(x_1, x_2)
        gh = grad_h(x_1, x_2)

        # Compute the gradient of the Lagrangian
        grad_lag_value = grad_lag(x_1, x_2, lamda)

        # Compute the Hessian of the Lagrangian
        hess_lag_value = hess_lag(x_1, x_2, lamda)

        # Build the augmented system matrix A and the right-hand side vector b for the Newton step
        # A is a block matrix combining the Hessian of the Lagrangian and the gradient of the constraint.
        A = np.block([
            [hess_lag_value, -gh.reshape(-1, 1)],  # First block: Hessian and negative constraint gradient
            [-gh, np.array([[0]])]  # Second block: Negative constraint gradient and a scalar zero
        ])

        # b is the vector representing the right-hand side of the system, combining the negative gradient of the Lagrangian and the constraint value h(x_1, x_2).
        b = np.concatenate([-grad_lag_value, [h(x_1, x_2)]])

        # Solve the linear system A * delta = b to find the Newton step (delta)
        delta = np.linalg.solve(A, b)

        # Update the variables (x_1, x_2, lambda) by moving along the Newton direction
        x_1 += alpha * delta[0]  # Update x_1
        x_2 += alpha * delta[1]  # Update x_2
        lamda += alpha * delta[2]  # Update lambda (Lagrange multiplier)

        # Check if the gradient of the Lagrangian is small enough to satisfy the convergence criterion
        if np.linalg.norm(grad_lag(x_1, x_2, lamda)) < eps:
            # If the norm of the gradient is less than the tolerance, stop the iterations
            break

        print('\nIterations: {}'.format(i))
        print('\nx = (x_1, x_2) = ({0:.5f}, {1:.5f}), lamda = {2:.5f}'.format(x_1, x_2, lamda))

    # Return the final values of x_1, x_2, and lambda
    return x_1, x_2, lamda


In [ ]:
x_1, x_2, lamda = -1, 1, -1
result = solveNewtonBased(x_1, x_2, lamda, alpha=1, eps=1e-5, MAX_ITER=100)

print("\nObtained Result using Newton:", result)

Starting point x0 = [-1, 1]

Iterations: 0

x = (x_1, x_2) = (-0.77423, 0.72577), lamda = -0.35104

Iterations: 1

x = (x_1, x_2) = (-0.74865, 0.66614), lamda = -0.21606

Obtained Result using Newton: (-0.7483381762503777, 0.663323446868971, -0.21232390186241443)


We can see that the results are the same with the ones in the Lab4 PDF and we obtain them with 2 iterations.



# 2. This basic iteration also has drawbacks, leading to a number of vital questions. It is a Newtonlike iteration, and thus may diverge from poor starting points. In our example we have started from a point that is near to the optimal solution. Try to perform some experiments with starting points that are farther away of the optimal solution.

In [ ]:
# Larger ranges for starting points
farther_ranges = [ (-8, -4), (-4, -1), (2, 8),]

far_away_points = np.array([
    np.random.uniform(low, high, 3) for low, high in farther_ranges
])

print(far_away_points)

[[-5.26550745 -7.91707189 -7.98360989]
 [-1.4286586  -1.8692028  -1.91740064]
 [ 2.17548553  2.41340769  6.42645673]]


In [ ]:

for point in far_away_points:
    x_1, x_2, lamda = point
    print('Starting point:\n\nx = (x_1, x_2) = ({0:.5f}, {1:.5f}), lamda = {2:.5f}\n'.format(x_1, x_2, lamda))
    solveNewtonBased(x_1, x_2, lamda)


Starting point:

x = (x_1, x_2) = (-5.26551, -7.91707), lamda = -7.98361

Starting point x0 = [-5.265507448361122, -7.91707189307672]

Iterations: 0

x = (x_1, x_2) = (2.84834, -7.66707), lamda = -12.30229

Iterations: 1

x = (x_1, x_2) = (-8.04633, -7.41707), lamda = -85829.04245

Iterations: 2

x = (x_1, x_2) = (-0.89725, -7.16707), lamda = -76258.25987

Iterations: 3

x = (x_1, x_2) = (25.62193, -6.91707), lamda = -2253914.70324

Iterations: 4

x = (x_1, x_2) = (25.28860, 42.68832), lamda = -59249653932822.30469

Iterations: 5

x = (x_1, x_2) = (24.95527, 14.06285), lamda = -39730996267044.75000

Iterations: 6

x = (x_1, x_2) = (24.62193, -14.48373), lamda = -80651091480492.07812

Iterations: 7

x = (x_1, x_2) = (24.28859, 13.08524), lamda = -218518191340249725286744064.00000

Iterations: 8

x = (x_1, x_2) = (23.95520, -15.34236), lamda = -474729432429685031563165696.00000

Iterations: 9

x = (x_1, x_2) = (23.61964, 10.47384), lamda = -6786748268420469466503053312.00000

Iterations:

It is obvious that now, using far away points from the optimal solution, the method cannot find the optimal solution. Leading us to the result that the method we already have is not efficint with points that are far away from the optimal solution.

# 3. One way to find the optimal solution from points that are far away of the optimal solution is to start the optimization with another function that allows us to find an approximation to the solution we are looking for. Once an approximate solution is found, we can apply the Newton-based technique we have presented previously to find the optimal solution.

# The function that allows us to find an approximation to the solution we are looking for is called, in this context, the merit function. Usually, a merit function is the sum of terms that include the objective function and the amount of infeasibility of the constraints. One example of a merit function for the problem we are treating is the quadratic penalty function  where  is some positive number. The greater the value of ρ, the greater the penalty for infeasibility. The difficulty arises in defining a proper merit function for a particular equality constrained problem.

# Here we propose you to take  and perform a classical gradient descent (with backtraking if you want) to find and approximation to the solution we are looking for. Observe if you arrive near to the optimal solution of the problem.Take into account that you may have numerical problems with the gradient. A simple way to deal with it is to normalize the gradient at each iteration, , and use this normalized gradient as search direction.

Initially we need to define a function that evaluates the merti function and one for the gradient of the merit. Following we define a function for gradient descent and we are using the gradient normalization to avoid numerical problems.

In [ ]:
def merit(x_1, x_2, ro=10):
    return f(x_1, x_2) + ro * h(x_1, x_2) ** 2  # Objective + penalty for constraint violation

def grad_merit(x_1, x_2, ro=10):
    return grad_f(x_1, x_2) + 2 * ro * h(x_1, x_2) * grad_h(x_1, x_2)  # Gradient of f + penalty gradient

# Gradient Descent Algorithm: Minimizes the objective function using the gradient descent method
# with an adaptive step size based on function evaluations.
def gradient_descent(f, grad_f, w0, f_tol=1e-3, grad_tol=1e-5):
    x = [w0]  # Store the history of points visited during the optimization

    while True:
        # Calculate the gradient at the current point
        gradient_of_f = grad_f(w0[0], w0[1])

        # Normalize the gradient
        grad_normalized = gradient_of_f / np.linalg.norm(gradient_of_f)

        # Initialize step size to 1
        alpha = 1

        #  Reduce alpha until the objective function decreases
        while f(*(w0 - alpha * grad_normalized)) >= f(*w0):
            alpha /= 2  # Halve alpha until the condition is satisfied

        # Update the current point
        w0 = w0 - alpha * grad_normalized

        # Append the updated point to the history
        x.append(w0)

        #  If the change in function value is smaller than the tolerance, stop.
        #  If the gradient's norm is smaller than the gradient tolerance, stop.
        gradient_of_f = grad_f(w0[0], w0[1])
        grad_normalized = gradient_of_f / np.linalg.norm(gradient_of_f)
        if np.abs(f(*x[-1]) - f(*x[-2])) < f_tol or np.linalg.norm(grad_normalized) < grad_tol:
            return np.array(x)  # Return the history of points visited


We are now using the same 3 random points that are far away from the optimal to test this method using the gradient descent and the merit functions wth normalization for gradient.

In [ ]:
for point in far_away_points:
    solution_approx = gradient_descent(merit, grad_merit, point[:2])
    print("Approximation:", solution_approx[-1])

Approximation: [-0.88379466  0.32295731]
Approximation: [-0.82317412  0.54968233]
Approximation: [-0.83338108  0.53404357]


Using this method, even with points far away from the optimal (same points we used before) our results are now closer to the optimal solution but not the same. They are like an approximation of the solution.

# 4. As previously commented, the minimizers of the merit function  do not necessarily have to coincide with the minimizers of the constrained problem. Thus, once we “sufficiently” approach the optimal solution we may use the Newton method (with ) to find the solution to the problem.

# Therefore the algorithm consists in starting with the Merit function to obtain an approximation to the optimal point we are looking for. Once an approximation to the solution is found,use the Newton-based method to find the optimal solution. Check if you are able to find the optimal solution to the problem.

Using the last approximation obtained from merit and gradient descent, we are testing if Newtons method can find the optimal solution from now on.


In [ ]:
solution_x = solveNewtonBased(solution_approx[-1,0], solution_approx[-1,1], lamda=-1, alpha=1)


Starting point x0 = [-0.833381079215307, 0.5340435724734561]

Iterations: 0

x = (x_1, x_2) = (-0.80261, 0.60105), lamda = -0.19829

Iterations: 1

x = (x_1, x_2) = (-0.75282, 0.66301), lamda = -0.20565

Iterations: 2

x = (x_1, x_2) = (-0.74832, 0.66336), lamda = -0.21228


We can see that using both methods combined we are now able to find the optimal solution.